# Import libraries

In [ ]:
# !pip install shap

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
from thefuzz import process
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from patsy import dmatrix
import shap
import warnings
warnings.filterwarnings('ignore')

# Create the network

In [ ]:
data = pd.read_csv('network_data.csv', index_col = 0)
df = data.groupby(['source', 'target']).size().reset_index(name = 'weight')
df = data.merge(df, on = ['source', 'target'], how = 'left')

In [ ]:
G = nx.from_pandas_edgelist(df = df,
                            source = 'source',
                            target = 'target',
                            edge_attr = 'weight',
                            create_using = nx.DiGraph()
                            )
G.number_of_nodes(), G.number_of_edges()

# Data for Regression Models

In [ ]:
nodes = list(G.nodes())

in_deg_cen = nx.in_degree_centrality(G)
out_deg_cen = nx.out_degree_centrality(G)
bet_cen = nx.betweenness_centrality(G)
clo_cen = nx.closeness_centrality(G)
eig_cen = nx.eigenvector_centrality(G)
clu_coef = nx.clustering(G).values()

In [ ]:
model_data = pd.DataFrame(columns = ['avg_ranking', 'in_degree_centrality', 'out_degree_centrality', 
                                     'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality',
                                     'clustering_coefficient'], 
                          index = nodes)
model_data['in_degree_centrality'] = in_deg_cen
model_data['out_degree_centrality'] = out_deg_cen
model_data['betweenness_centrality'] = bet_cen
model_data['closeness_centrality'] = clo_cen
model_data['eigenvector_centrality'] = eig_cen
model_data['clustering_coefficient'] = clu_coef
model_data

#### Load leagues standings

In [ ]:
england = pd.read_csv('england_standings.csv', index_col = 0)
italy = pd.read_csv('italy_standings.csv', index_col = 0)
france = pd.read_csv('france_standings.csv', index_col = 0)
netherlands = pd.read_csv('netherlands_standings.csv', index_col = 0)
portugal = pd.read_csv('portugal_standings.csv', index_col = 0)
russia = pd.read_csv('russia_standings.csv', index_col = 0)
spain = pd.read_csv('spain_standings.csv', index_col = 0)
germany = pd.read_csv('germany_standings.csv', index_col = 0)

#### Compute the average ranking of each team over time

In [ ]:
results = []

leagues = [england, italy, france, netherlands, portugal, russia, spain, germany]

for league in leagues:
    # Initialize an empty dictionary to store positions
    positions = {team: [] for team in league.stack().unique()}

    # Iterate over each column -> seasons and each row -> team positions
    for season in league.columns:
        for position, team in enumerate(league[season], start = 1):
            if not pd.isna(team):
                positions[team].append(position)

    # Calculate the number of seasons each team is present in
    num_seasons_present = {team: len(positions[team]) for team in positions}

    # Determine the maximum number of seasons present in the league
    max_seasons = max(num_seasons_present.values())

    # Fill missing seasons with NaN values
    for team, positions_list in positions.items():
        num_missing_seasons = max_seasons - num_seasons_present[team]
        if num_missing_seasons > 0:
            positions[team] += [np.nan] * num_missing_seasons

    # Convert the dictionary to a DataFrame
    positions_df = pd.DataFrame(positions)

    # Calculate the average position for each team
    average_positions = positions_df.mean(axis = 0, skipna = True)

    # Create a DataFrame for average positions
    average_positions_df = pd.DataFrame({'Team': positions_df.columns, 'Average Position': average_positions}).reset_index(drop = True)

    # Sort the DataFrame by average position
    average_positions_df = average_positions_df.sort_values(by = 'Average Position')
    
    results.append(average_positions_df)
    
standings = pd.DataFrame()

for el in results:
    standings = pd.concat([standings, el], ignore_index = True)
    
standings

In [ ]:
for team in model_data.index:
    rank = standings.loc[standings['Team'] == team, 'Average Position']
    model_data.loc[team, 'avg_ranking'] = rank.iloc[0]

model_data['avg_ranking'] = pd.to_numeric(model_data['avg_ranking'], errors = 'coerce')
model_data['in_degree_centrality'] = pd.to_numeric(model_data['in_degree_centrality'], errors = 'coerce')
model_data['out_degree_centrality'] = pd.to_numeric(model_data['out_degree_centrality'], errors = 'coerce')
model_data['betweenness_centrality'] = pd.to_numeric(model_data['betweenness_centrality'], errors = 'coerce')
model_data['closeness_centrality'] = pd.to_numeric(model_data['closeness_centrality'], errors = 'coerce')
model_data['eigenvector_centrality'] = pd.to_numeric(model_data['eigenvector_centrality'], errors = 'coerce')
model_data['clustering_coefficient'] = pd.to_numeric(model_data['clustering_coefficient'], errors = 'coerce')

# Final data 
model_data

# Regression Models

### OLS Model

In [ ]:
X = model_data[['in_degree_centrality', 'out_degree_centrality', 
                'betweenness_centrality', 'closeness_centrality', 
                'eigenvector_centrality', 'clustering_coefficient']]
y = model_data['avg_ranking']

X = np.asarray(X)
y = np.asarray(y)

X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model = sm.OLS(y_train, X_train).fit()
print(model.summary())
print(' ')

# Predict and calculate RMSE for train and test sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_rmse = np.sqrt(np.mean((y_train - y_train_pred) ** 2))
test_rmse = np.sqrt(np.mean((y_test - y_test_pred) ** 2))

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')
print(' ')

# Residuals plot
residuals = y_train - y_train_pred
plt.figure(figsize = (7, 5))
sns.residplot(x = y_train_pred, y = residuals, lowess = True, line_kws = {'color': 'r', 'ls': '--'})
plt.ylim((-10, 10))
plt.xlim((0, 21))
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('OLS Residual Plot')
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.show()

# Q-Q plot for normality of residuals
fig = plt.figure(figsize = (7, 5))
ax = fig.add_subplot(111)
sm.qqplot(residuals, ax = ax)
plt.ylim((-10, 7))
plt.xlim((-10, 7))
ax.set_aspect('equal', adjustable='box')
lim = np.array(ax.get_xlim())
ax.plot(lim, lim, 'r--', lw=2)
plt.title('OLS Q-Q Plot')
plt.show()

### OLS + Interaction terms

In [ ]:
# Adding interaction terms
interaction_model_data = dmatrix('in_degree_centrality + out_degree_centrality + \
                                  betweenness_centrality + closeness_centrality + \
                                  eigenvector_centrality + clustering_coefficient + \
                                  in_degree_centrality:out_degree_centrality + \
                                  in_degree_centrality:eigenvector_centrality + \
                                  out_degree_centrality:eigenvector_centrality', data = model_data, return_type = 'dataframe')
y = model_data['avg_ranking']

X = np.asarray(interaction_model_data)
y = np.asarray(y)

X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model = sm.OLS(y_train, X_train).fit()
print(model.summary())
print(' ')

# Predict and calculate RMSE for train and test sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_rmse = np.sqrt(np.mean((y_train - y_train_pred) ** 2))
test_rmse = np.sqrt(np.mean((y_test - y_test_pred) ** 2))

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')
print(' ')

# Residuals plot
residuals = y_train - y_train_pred
plt.figure(figsize = (7, 5))
sns.residplot(x = y_train_pred, y = residuals, lowess = True, line_kws = {'color': 'r', 'ls': '--'})
plt.ylim((-10, 10))
plt.xlim((0, 21))
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('OLS + Interactions Residual Plot')
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.show()

# Q-Q plot for normality of residuals
fig = plt.figure(figsize = (7, 5))
ax = fig.add_subplot(111)
sm.qqplot(residuals, ax = ax)
plt.ylim((-10, 7))
plt.xlim((-10, 7))
ax.set_aspect('equal', adjustable='box')
lim = np.array(ax.get_xlim())
ax.plot(lim, lim, 'r--', lw=2)
plt.title('OLS + Interactions Q-Q Plot')
plt.show()

### Decision Tree, Random Forest, Gradient Boosting, Support Vector Regression and Neural Network

In [ ]:
X = model_data[['in_degree_centrality', 'out_degree_centrality', 
                'betweenness_centrality', 'closeness_centrality', 
                'eigenvector_centrality', 'clustering_coefficient']]
y = model_data['avg_ranking']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Models and their RMSEs
models = {
    'Regression Tree': DecisionTreeRegressor(max_depth = 5, random_state = 42),
    'Random Forest': RandomForestRegressor(n_estimators = 100, max_depth = 10, random_state = 42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.1, max_depth = 5, random_state = 42),
    'Support Vector Regression': SVR(kernel = 'rbf', C = 1e3, gamma = 0.1),
    'Neural Network': MLPRegressor(hidden_layer_sizes = (100, 100), max_iter = 1000, random_state = 42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_rmse = np.sqrt(np.mean((y_train - y_train_pred) ** 2))
    test_rmse = np.sqrt(np.mean((y_test - y_test_pred) ** 2))
    print(f'Model: {name}')
    print(f'Train RMSE: {train_rmse}')
    print(f'Test RMSE: {test_rmse}')
    
    residuals = y_train - y_train_pred
    plt.figure(figsize = (7, 5))
    sns.residplot(x = y_train_pred, y = residuals, lowess = True, line_kws = {'color': 'r', 'ls': '--'})
    plt.ylim((-10, 10))
    plt.xlim((0, 21))
    plt.xlabel('Fitted Values')
    plt.ylabel('Residuals')
    plt.title(f'{name} Residual Plot')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.show()
    
    fig = plt.figure(figsize = (7, 5))
    ax = fig.add_subplot(111)
    sm.qqplot(residuals, ax = ax)
    plt.ylim((-10, 7))
    plt.xlim((-10, 7))
    ax.set_aspect('equal', adjustable='box')
    lim = np.array(ax.get_xlim())
    ax.plot(lim, lim, 'r--', lw=2)
    plt.title(f'{name} Q-Q Plot')
    plt.show()

### Focus on Random Forest

In [ ]:
X = model_data[['in_degree_centrality', 'out_degree_centrality', 
                'betweenness_centrality', 'closeness_centrality', 
                'eigenvector_centrality', 'clustering_coefficient']]
y = model_data['avg_ranking']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Random Forest Model
model = RandomForestRegressor(n_estimators = 100, max_depth = 10, random_state = 42).fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_rmse = np.sqrt(np.mean((y_train - y_train_pred) ** 2))
test_rmse = np.sqrt(np.mean((y_test - y_test_pred) ** 2))

# Get feature importances
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X.columns

# Plot the feature importances
plt.figure(figsize = (7, 5))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices], align = "center", zorder = 3)
plt.xticks(range(X.shape[1]), feature_names[indices], rotation = 90)
plt.ylabel('Importance')
plt.yticks([.10, .20, .30, .40])    
plt.xlim([-1, X.shape[1]])
plt.grid(which = 'major', axis = 'y', zorder = 0, alpha = 0.5)
plt.show()

In [ ]:
# Create a SHAP explainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# Summary plot
shap.summary_plot(shap_values, X, feature_names = feature_names, plot_type = 'violin')